# Purpose

### 2023-05-25

Update the labels for the r/zelda PN! Add them to the existing table so we can use them for model refresh.


# Imports & notebook setup

In [1]:
%load_ext google.colab.data_table
%load_ext autoreload
%autoreload 2

In [2]:
# colab auth for BigQuery, google drive, & google sheets (gspread)
from google.colab import auth, files, drive
from google.auth import default


auth.authenticate_user()
print('Authenticated')

Authenticated


### General Imports


In [3]:
# Regular Imports
from datetime import datetime
import logging
import os
import string

from google.cloud import bigquery
from tqdm import tqdm
import pandas as pd
import gspread


logger = logging.getLogger(__name__)


# auth for google sheets
gc = gspread.authorize(default()[0])


# Set env variable needed by some libraries to get data from BigQuery. 
os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-prod-165221'

# Official `gspread` Documentation
See here for official documentation: https://docs.gspread.org/en/latest/user-guide.html

---


### Using gspread with pandas
pandas is a popular library for data analysis. The simplest way to get data from a sheet to a pandas DataFrame is with `get_all_records()`:
```python
import pandas as pd

dataframe = pd.DataFrame(worksheet.get_all_records())
```

Here’s a basic example for writing a dataframe to a sheet. With update() we put the header of a dataframe into the first row of a sheet followed by the values of a dataframe:

```python
import pandas as pd

worksheet.update([dataframe.columns.values.tolist()] + dataframe.values.tolist())
```



# Define helper functions

### Open or Create Sheet
These helper functions help creating or opening existing sheets or worksheets (tabs).

The functions will try to open an existing sheet by:
- `GSHEET_KEY` first
- `GSHEET_NAME` second

If it can't find an existing sheet by key or name, it'll create a new one.

### Open or create tabs (worksheets)
Given a list of worksheet names, this function will open (if they exist) or create a new worksheet with the input names.

In [4]:
def open_or_create_sheet(
    gsheet_name: str,
    gsheet_key: str = None,
):
    """Open or create an existing google sheet"""
    # create worksheet or read existing
    if GSHEET_KEY is not None:
        sh = gc.open_by_key(GSHEET_KEY)
        print(f"Opening google worksheet: {sh.title} ...")
    else:
        try:
            # try to open by name:
            sh = gc.open(GSHEET_NAME)
            print(f"Opening google worksheet: {sh.title} ...")
            print(f"*** Sheet ID (assign it to GSHEET_KEY variable): ***\n{sh.id}\n")
        except Exception as e:
            print(f"** Creating google worksheet: {GSHEET_NAME} ...")
            sh = gc.create(GSHEET_NAME)
            print(f"\n*** Sheet ID (assign it to GSHEET_KEY variable): ***\n{sh.id}\n")
    print(f"Google sheet URL:\nhttps://docs.google.com/spreadsheets/d/{sh.id}")
    return sh


def open_or_create_worksheets(
    sheet_object,
    worksheet_names: list,
) -> dict:
    """Open or create worksheets in input sheet.
    Return a dictionary with worksheet objects
    """
    d_output = dict()

    for wsh_name in worksheet_names:
        try:
            d_output[wsh_name] = sheet_object.worksheet(wsh_name)
            print(f"  Opening tab/sheet: {wsh_name}")
        except Exception as e:
            print(f"  ** Creating tab/sheet: {wsh_name}")
            d_output[wsh_name] = sheet_object.add_worksheet(wsh_name, rows=5, cols=5)
    
    # get & return existing worksheets
    for wsh_ in sheet_object.worksheets():
        wsh_title = wsh_.title
        print(f"  Opening tab/sheet: {wsh_title}")
        d_output[wsh_title] = wsh_

    return d_output



# Get sheet with metadata for one-off PNs.

I (David) created this sheet to clean up data needed from the one-off sending tool here:
- https://channels-dashboard.kubernetes.ue1.snooguts.net/one_offs?page=5

## Load raw sheet

In [5]:
%%time

from google.auth import default
import gspread


# Auth for bigquery & google sheets
auth.authenticate_user()
print('Authenticated')
gc = gspread.authorize(default()[0])


# You can use f-strings to generate the sheet name based on input variables
GSHEET_NAME = f'PN tables & send dates for model training'

# Set to None if we want a new sheet.
#   if you want to open an existing one, assign string, like:
#   '1kiiuOqHPJ5chV3zmx4HsXfpOGzO0QcuxW0uHTc85Peg'
GSHEET_KEY = '1kiiuOqHPJ5chV3zmx4HsXfpOGzO0QcuxW0uHTc85Peg'

sh = open_or_create_sheet(
    gsheet_name=GSHEET_NAME,
    gsheet_key=GSHEET_KEY,
)

l_worksheet_names = []

d_worksheets = open_or_create_worksheets(
    sheet_object=sh,
    worksheet_names=l_worksheet_names,
)

Authenticated
Opening google worksheet: PN tables & send dates for model training ...
Google sheet URL:
https://docs.google.com/spreadsheets/d/1kiiuOqHPJ5chV3zmx4HsXfpOGzO0QcuxW0uHTc85Peg
  Opening tab/sheet: raw_tables_and_dates
  Opening tab/sheet: Pivot Table 1
  Opening tab/sheet: README
  Opening tab/sheet: zelda PN estimates
CPU times: user 49.5 ms, sys: 7.67 ms, total: 57.2 ms
Wall time: 1.26 s


In [6]:
%%time
df_campaigns_train_raw = pd.DataFrame(d_worksheets['raw_tables_and_dates'].get_all_records())
df_campaigns_train_raw = df_campaigns_train_raw.rename(columns={c: c.replace(' ', '_') for c in df_campaigns_train_raw.columns})
print(df_campaigns_train_raw.shape)

(39, 12)
CPU times: user 7.18 ms, sys: 2.03 ms, total: 9.21 ms
Wall time: 282 ms


In [7]:
df_campaigns_train_raw.head()

,date_sent_utc,full_table_name,train_v0,train_v1,day_of_week_sent,date_training,target_country,Expected_Sends,table_name_only,date_training_(extra_lag),CTR_check,
0,12/1/2022,reddit-growth-prod.generated_one_offs.20221201...,,,Thursday,11/30/2022,gb,"74,883",20221201174412_vnicoleheard_gb_taskmasterstand...,,,
1,12/2/2022,reddit-growth-prod.generated_one_offs.20221201...,,,Friday,12/1/2022,ca,"78,986",20221201193322_elizabethpollard_ca_canada_this...,,,
2,12/2/2022,reddit-growth-prod.generated_one_offs.20221201...,1,0,Friday,12/1/2022,de,"32,358",20221201193812_elizabethpollard_de_de_der_topb...,,,
3,12/2/2022,reddit-growth-prod.generated_one_offs.20221201...,,,Friday,12/1/2022,ca,"78,986",20221201193322_elizabethpollard_ca_canada_this...,,,
4,12/14/2022,reddit-growth-prod.generated_one_offs.20221214...,,,Wednesday,12/13/2022,fr,"93,361",20221214012134_elizabethpollard_fr_ligue1world...,,,


## Filter to only pre-selected campaigns

In [10]:
df_campaigns_train = (
    df_campaigns_train_raw
    [df_campaigns_train_raw['full_table_name'] == 'reddit-employee-datasets.sahil_verma.totk_pn_ml_targeting_20230512']
    .copy()
)
df_campaigns_train

,date_sent_utc,full_table_name,train_v0,train_v1,day_of_week_sent,date_training,target_country,Expected_Sends,table_name_only,date_training_(extra_lag),CTR_check,
38,5/12/2023,reddit-employee-datasets.sahil_verma.totk_pn_m...,1,0,Friday,5/11/2023,"MX, GB, DE","38,000",totk_pn_ml_targeting_20230512,,https://app.mode.com/reddit/reports/7e4911e117...,


In [11]:
# from time import strftime
## Convert dates to ISO standard that SQL queries should understand better
for dt_ in ['date_sent_utc', 'date_training']:
    df_campaigns_train[dt_] = (
        pd.to_datetime(
            df_campaigns_train[dt_]
        )
        .dt.strftime("%Y-%m-%d")
    ) 

In [12]:
df_campaigns_train

,date_sent_utc,full_table_name,train_v0,train_v1,day_of_week_sent,date_training,target_country,Expected_Sends,table_name_only,date_training_(extra_lag),CTR_check,
38,2023-05-12,reddit-employee-datasets.sahil_verma.totk_pn_m...,1,0,Friday,2023-05-11,"MX, GB, DE","38,000",totk_pn_ml_targeting_20230512,,https://app.mode.com/reddit/reports/7e4911e117...,


In [13]:
# convert to dictionary where I can use the strings to query the right data:

l_cols_for_extraction = [
    'date_sent_utc',
    'full_table_name',
]
l_campaign_pt_and_table = df_campaigns_train[l_cols_for_extraction].to_dict(orient='records')
print(len(l_campaign_pt_and_table))
l_campaign_pt_and_table

1


[{'date_sent_utc': '2023-05-12',
  'full_table_name': 'reddit-employee-datasets.sahil_verma.totk_pn_ml_targeting_20230512'}]

# Create SQL query to create or insert data into training table

General set up:
- Declare variables (e.g., PT window start & end)
- CREATE or INSERT statement
- SELECT statement for data to insert (CTEs & other)

NOTE: we need to add `r` before the string to show that it's a `raw` string -- we don't want it to mess with the regexes in the SQL queries!!

## Define SQL queries

In [17]:
#@title
train_data_table = 'reddit-employee-datasets.david_bermejo.pn_training_data_20230515'

SQL_DEFINE_VARS = r"""
-- Get labels for receives & clicks

-- Only look at click events 5 days afert send
DECLARE PT_WINDOW_START DATE DEFAULT DATE("${date_sent_utc}");
DECLARE PT_WINDOW_END DATE DEFAULT PT_WINDOW_START + 5;

DECLARE RX_GET_SUBREDDIT_NAME STRING DEFAULT r"(?i)\br\/([a-zA-Z0-9]\w{2,30}\b)";
"""

SQL_CREATE_TABLE = r"""
CREATE TABLE `${train_data_table}`
CLUSTER BY pn_id
AS (
"""

SQL_INSERT_INTO_TABLE = r"""
-- Delete data we're trying to re-insert
-- Should be faster with new pn_id & clustering
--   Note that in the custom tables the column is `title`, but in `pn_sends` it is `notification_title`
DELETE
    `${train_data_table}`
WHERE
    pn_id = (
        SELECT DISTINCT
        CONCAT(
            CAST(PT_WINDOW_START AS STRING)
            , "-"
            , title
            , "-"
            , deeplink_uri
        ) AS pn_id
        FROM `${full_table_name}`
    )
;


-- Insert latest data
INSERT INTO `${train_data_table}`
(
"""

SQL_SELECT_DATA = r"""
WITH
send_long AS (
    SELECT DISTINCT
        a.correlation_id
        , notification_title
        , notification_type
        , deeplink_uri
        , REGEXP_EXTRACT(deeplink_uri, RX_GET_SUBREDDIT_NAME, 1) AS target_subreddit
        , a.user_id
        , a.app_name
        , b.device_id

    FROM `data-prod-165221.channels.pn_sends` a
        INNER JOIN  `${full_table_name}` b
        ON a.user_id = b.user_id
            AND a.notification_title = b.title
            AND a.notification_type = b.campaign_type
    WHERE 1=1
        AND DATE(a.pt) = PT_WINDOW_START
    -- GROUP BY 1,2,3,4,5,6,7
)
, send_wide AS (
    SELECT
        correlation_id
        , user_id
        , notification_title
        , notification_type
        , target_subreddit
        , deeplink_uri

        , 1 AS send
        # , ARRAY_CONCAT_AGG(DISTINCT device_id) AS device_ids
        , COUNT(correlation_id) AS send_count
        , COUNT(DISTINCT (CASE WHEN app_name = 'ios' THEN correlation_id
            ELSE NULL
        END
        )) AS send_ios
        , COUNT(DISTINCT (CASE WHEN app_name = 'android' THEN correlation_id
            ELSE NULL
        END
        )) AS send_android
        , COUNT(CASE WHEN app_name NOT IN ('android', 'ios') THEN correlation_id
            ELSE NULL
        END
        ) AS send_other

    FROM send_long
    GROUP BY 1,2,3,4,5,6
)
, receive_long as (
    SELECT
        a.correlation_id
        , a.user_id
        , a.app_name
        -- If ANY receive was supressed, count ALL as supressed
        , SUM(IF(a.is_suppressed = True, 1, 0)) AS supressed_count
    FROM `data-prod-165221.channels.pn_receives` AS a
        INNER JOIN send_wide AS b
        ON a.user_id = b.user_id
            AND a.correlation_id = b.correlation_id
    WHERE 1=1
        AND DATE(pt) between PT_WINDOW_START and PT_WINDOW_END

    GROUP BY 1,2,3
)
, receive_wide AS (
    SELECT
        user_id
        , correlation_id

        , 1 AS receive
        , COUNT(correlation_id) AS receive_count
        , SUM(supressed_count) AS suppressed_count

        , COUNT(DISTINCT (CASE WHEN app_name = 'ios' THEN correlation_id
            ELSE NULL
        END)) AS receive_ios
        , COUNT(DISTINCT (CASE WHEN app_name = 'android' THEN correlation_id
            ELSE NULL
        END)) AS receive_android
        , COUNT(CASE WHEN app_name NOT IN ('android', 'ios') THEN correlation_id
            ELSE NULL
        END
        ) AS receive_other

        , COUNT(
            DISTINCT(
                CASE WHEN app_name = 'ios' AND supressed_count >= 1 THEN correlation_id
                ELSE NULL
                END
            )
        ) AS suppressed_ios
        , COUNT(
            DISTINCT(
                CASE WHEN app_name = 'android' AND supressed_count >= 1 THEN correlation_id
                ELSE NULL
                END
            )
        ) AS suppressed_android

    FROM receive_long
    GROUP By 1,2
)
, click_long as (
    SELECT
        a.correlation_id,
        a.user_id,
        a.app_name
    FROM `data-prod-165221.channels.pn_clicks` AS a
        INNER JOIN send_wide AS b
        ON a.user_id = b.user_id
            AND a.correlation_id = b.correlation_id
    WHERE 1=1
        AND DATE(pt) between PT_WINDOW_START and PT_WINDOW_END

    GROUP BY 1,2,3
)
, click_wide AS (
SELECT
    user_id
    , correlation_id

    , 1 AS click
    , COUNT(correlation_id) AS click_count

    , COUNT(DISTINCT (CASE WHEN app_name = 'ios' THEN correlation_id
        ELSE NULL
    END)) AS click_ios
    , COUNT(DISTINCT (CASE WHEN app_name = 'android' THEN correlation_id
        ELSE NULL
    END)) AS click_android
    , COUNT(CASE WHEN app_name NOT IN ('android', 'ios') THEN correlation_id
        ELSE NULL
    END
    ) AS click_other
FROM click_long
GROUP BY 1,2
)
, all_data_wide AS (
    SELECT
        -- Note that we're getting the target_subreddit from the deeplink URI
        s.correlation_id
        , s.user_id
        , s.target_subreddit
        , s.notification_title
        , s.notification_type
        , s.send
        , r.receive
        , c.click
        , CASE
            WHEN (
                r.receive = 1
                -- Keep receives when we have more receives in (android & iOS) than suppressed_receives
                AND (
                    (COALESCE(receive_ios, 0) + COALESCE(receive_android, 0)) >
                    (COALESCE(suppressed_ios, 0) + COALESCE(suppressed_android, 0))
                )
            ) THEN 1
            WHEN r.receive IS NOT NULL THEN 0
            ELSE NULL
        END AS receive_not_suppressed


        , c.* EXCEPT(correlation_id, user_id, click)
        , r.* EXCEPT(correlation_id, user_id, receive)
        , s.* EXCEPT(correlation_id, user_id, notification_title, notification_type, send, target_subreddit)

    FROM send_wide AS s
        LEFT JOIN receive_wide AS r
            ON s.user_id = r.user_id AND s.correlation_id = r.correlation_id
        LEFT JOIN click_wide AS c
            ON s.user_id = c.user_id AND s.correlation_id = c.correlation_id
)

-- Final select for TABLE
SELECT 
    PT_WINDOW_START AS pt_send
    -- Create new campaign-id column so it's easier to find & delete campaigns
    , CONCAT(
        CAST(PT_WINDOW_START AS STRING)
        , "-"
        , notification_title
        , "-"
        , deeplink_uri
    ) AS pn_id
    , *
FROM all_data_wide
);  -- Close CREATE TABLE parens
"""

#  replace escape character b/c we sometimes need to use it with
#   regex or in JSON_EXTRACT() function
SQL_FULL_CREATE = (
    SQL_DEFINE_VARS + SQL_CREATE_TABLE + SQL_SELECT_DATA
    .replace("$.", "$$.")
    .replace("$|", "$$|")
    .replace('$"', '$$"')
)

SQL_FULL_INSERT = (
    SQL_DEFINE_VARS + SQL_INSERT_INTO_TABLE + SQL_SELECT_DATA
    .replace("$.", "$$.")
    .replace("$|", "$$|")
    .replace('$"', '$$"')
)

## Run queries for campaigns selected for v1-train 

In [18]:
log_query = True
bigquery_client = bigquery.Client()

for d_table_ in tqdm(l_campaign_pt_and_table):
    try:
        bigquery_client.get_table(train_data_table) 
        print("Table {} already exists.".format(train_data_table))
        template = string.Template(SQL_FULL_INSERT)
    except Exception as e:
        print("Table {} is NOT found.".format(train_data_table))
        template = string.Template(SQL_FULL_CREATE)
        
    sql = template.substitute(
        **d_table_,
        **{'train_data_table': train_data_table}
    )
    if log_query:
        print(sql)

    print(f"Running query for params:...\n  {d_table_}")
    query_start_time = datetime.utcnow()
    print(f"  {query_start_time} | query START time")

    query_job = bigquery_client.query(sql)
    query_job.result()
    query_end_time = datetime.utcnow()
    print(f"  {query_end_time} | query END time")
    print(f"  {query_end_time - query_start_time} | query ELAPSED time")

  0%|          | 0/1 [00:00<?, ?it/s]

Table reddit-employee-datasets.david_bermejo.pn_training_data_20230515 already exists.

-- Get labels for receives & clicks

-- Only look at click events 5 days afert send
DECLARE PT_WINDOW_START DATE DEFAULT DATE("2023-05-12");
DECLARE PT_WINDOW_END DATE DEFAULT PT_WINDOW_START + 5;

DECLARE RX_GET_SUBREDDIT_NAME STRING DEFAULT r"(?i)\br\/([a-zA-Z0-9]\w{2,30}\b)";

-- Delete data we're trying to re-insert
-- This delete takes too long! almost as long as inserting the data! Should be faster with new pn_id & clustering
DELETE
    `reddit-employee-datasets.david_bermejo.pn_training_data_20230515`
WHERE
    -- Note that in the custom tables the column is `title`, but in `pn_sends` it is `notification_title`
    pn_id = (
        SELECT DISTINCT
        CONCAT(
            CAST(PT_WINDOW_START AS STRING)
            , "-"
            , title
            , "-"
            , deeplink_uri
        ) AS pn_id
        FROM `reddit-employee-datasets.sahil_verma.totk_pn_ml_targeting_20230512`
    

100%|██████████| 1/1 [00:28<00:00, 28.52s/it]

  2023-05-25 17:26:23.067763 | query END time
  0:00:27.823903 | query ELAPSED time


## Inspect output of tables

In [27]:
%%time
%%bigquery df_train_check1

SELECT 
    -- pt_send
    pn_id
    , SUM(send) AS send_total
    , SUM(receive) AS receive_total
    , SUM(click) AS click_total
    , SAFE_DIVIDE(SUM(receive), SUM(send)) AS receive_pct

    , SUM(receive_not_suppressed) AS receive_not_suppressed_total
    
    , SAFE_DIVIDE(SUM(click), SUM(receive)) AS ctr_receive
    , SAFE_DIVIDE(SUM(click), SUM(receive_not_suppressed)) AS ctr_receive_no_suppressed
FROM `reddit-employee-datasets.david_bermejo.pn_training_data_20230515` 
GROUP BY 1
ORDER BY send_total DESC
;

Query is running:   0%|          |

Downloading:   0%|          |

CPU times: user 125 ms, sys: 10 ms, total: 135 ms
Wall time: 2.19 s


In [29]:
(
    df_train_check1
    .sort_values(by=['receive_pct'], ascending=False)
)

,pn_id,send_total,receive_total,click_total,receive_pct,receive_not_suppressed_total,ctr_receive,ctr_receive_no_suppressed
14,2023-03-01-The wait is over-https://www.reddit...,6894,6458,311,0.936757,5047,0.048157,0.061621
11,2023-03-01-TheMandolorianTV:-https://www.reddi...,69098,64366,3904,0.931518,51236,0.060653,0.076196
9,2023-05-10-Vegemite on pizza: Y/N? 🤔-https://w...,72160,66482,2099,0.921314,51595,0.031572,0.040682
1,2023-05-10-Zelda + AI = 🤯-https://www.reddit.c...,154904,142498,11359,0.919912,108173,0.079713,0.105008
8,2023-04-18-Live! AMA mit Saidi Sulilatu 📈-http...,74027,67817,1448,0.916112,51759,0.021352,0.027976
12,2023-05-12-r/zelda is finally here!-https://ww...,49850,45616,3155,0.915065,42125,0.069164,0.074896
7,2023-02-20-Quer fugir do Carnaval?-https://www...,78447,71781,2373,0.915025,58368,0.033059,0.040656
13,2023-05-05-Diese Woche heiß 🔥-https://www.redd...,46912,42724,3089,0.910726,32931,0.072301,0.093802
6,"2023-05-05-Voyager AMA, douze points!-https://...",79253,72078,1506,0.909467,54407,0.020894,0.027680
2,2023-05-10-Live! AMA with Nikhil Chinapa-https...,109511,99571,1979,0.909233,75929,0.019875,0.026064


In [26]:
df_train_check1[[
    'send_total',
    'receive_total', 'receive_not_suppressed_total',
    'click_total'
]].sum()

send_total                      1372949
receive_total                   1242167
receive_not_suppressed_total     957529
click_total                       58772
dtype: int64

In [25]:
df_train_check1[[
    'receive_pct',
    'ctr_receive',
    'ctr_receive_no_suppressed'
]].mean()

receive_pct                  0.910057
ctr_receive                  0.046209
ctr_receive_no_suppressed    0.058467
dtype: float64

In [ ]:
42482 / 814830

0.052136028374016666